In [3]:
# !pip install pandas

In [2]:
# !pip install openai

In [4]:
# !export OPENAI_API_KEY=

### Augment dataset with GPT API

In [2]:
import pandas as pd
from openai import OpenAI
import json
import os 
import numpy as np

In [3]:
df = pd.read_json('data/ontology.json')[['id', 'name', 'description']]
df

In [5]:
def query_openai(sound_name, sound_description):
    """
    Use openai to classify sounds as sound effects or ambience.
    """
    client = OpenAI(api_key =os.environ.get('OPENAI_API_KEY'))
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON. Given the name of the sound and its description, you classify it as ambience (code: AMB) or sound effect (code: SFX). If it's a sound effect, you also include the object emitter. The schema of the JSON is always {'name': ---, 'type': --, 'object(s)': ---}"},
        {"role": "user", "content": "A sound effect is a sound that is directly emitted by an individual or by an object. For example, the sound of a car engine or a child screaming are sound effects. In contrast, an ambience sound cannot be directly attributed to an object, and it depends by the location. Examples of ambience sounds include a people talking in a busy street, a cafe, a hotel lobby etc."},
        {"role": "user", "content": f'Is this sound an ambience sound or sound effect? Name: {sound_name}. Description: {sound_description}.'}
          
      ]
    )
    return json.loads(response.choices[0].message.content)

In [1]:
classes = {'name': [], 'type': [], 'object(s)': []}

names = df.name.unique()
descriptions = df.description.unique()

for i in range(len(df)):
    response = query_openai(names[i], descriptions[i])
    print(response)
    
    classes['name'].append(names[i])
    
    if 'type' in response.keys():
        classification = response['type']
        classes['type'].append(classification)
    else:
        classes['type'].append(response)
        
    if 'object(s)' in response.keys():
        object = response['object(s)']
        classes['object(s)'].append(object)
    else:
        classes['object(s)'].append(response)

In [8]:
classes_df_output_3 = pd.DataFrame(classes)
classes_df_output_3

,name,type,object(s)
0,Human sounds,SFX,individual
1,Human voice,SFX,Human
2,Speech,SFX,Human
3,"Male speech, man speaking",SFX,human
4,"Female speech, woman speaking",SFX,woman
...,...,...,...
627,Recording,SFX,recorder/player
628,Field recording,AMB,
629,Gramophone record,SFX,Gramophone or vinyl record disc on a turntable
630,Compact disc,SFX,digital audio Compact Disc


I ran this 3 times and collected a dataset of outputs. Now, we'll generate the values by majority/agreement

### Post-processing (Majority voting)

In [15]:
df = pd.read_csv('extracted2.csv', index_col=0)

df['object(s)_y'] = df['object(s)_y'].str.split("object").str[-1].str.split("(s)").str[-1].str.split("'").str[-2] # nevermind this lol

df

Type (SFX vs Ambience): Majority wins:

In [26]:
df['MajorityType'] = np.where((df['type_x'] == 'SFX').astype(int) + (df['type_y'] == 'SFX').astype(int) + (df['type'] == 'SFX').astype(int) >= 2, 'SFX', 'AMB')

In [28]:
df = df.drop(columns=['type_x','type_y','type'])
df

,name,object(s)_x,object(s)_y,object(s),MajorityType
0,Human sounds,human body,Human body,individual,SFX
1,Human voice,Human,Human,Human,SFX
2,Speech,Human,Human,Human,SFX
3,"Male speech, man speaking",mouth of the adult male human,Human,human,SFX
4,"Female speech, woman speaking",human,woman,woman,SFX
...,...,...,...,...,...
627,Recording,NaN,Recording device,recorder/player,SFX
628,Field recording,NaN,N/A,NaN,AMB
629,Gramophone record,gramophone or vinyl record disc,c on turntable,Gramophone or vinyl record disc on a turntable,SFX
630,Compact disc,Compact Disc,c,digital audio Compact Disc,SFX


Sound emitter (object): Ask again GPT to find agreement

In [34]:
def openai_find_agreement(object_x, object_y, object_z):
    """
    Use openai to classify sounds as sound effects or ambience.
    """
    client = OpenAI(api_key =os.environ.get('OPENAI_API_KEY'))
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON. The JSON must contain the 'object' field. "},
        {"role": "user", "content": f"Three people described an object in three different ways. First person: {object_x}. Second person: {object_y}. Third person:{object_z}. What is the object?"}
          
      ]
    )
    return json.loads(response.choices[0].message.content)

In [35]:
objects_agreement = []
for row in df.iterrows():
    agreement = openai_find_agreement(row[-1]['object(s)_x'], row[-1]['object(s)_y'], row[-1]['object(s)'])
    objects_agreement.append(agreement['object'])

In [36]:
df['Object'] = objects_agreement
df

,name,object(s)_x,object(s)_y,object(s),MajorityType,Object
0,Human sounds,human body,Human body,individual,SFX,human body
1,Human voice,Human,Human,Human,SFX,Human
2,Speech,Human,Human,Human,SFX,Human
3,"Male speech, man speaking",mouth of the adult male human,Human,human,SFX,An adult male human
4,"Female speech, woman speaking",human,woman,woman,SFX,woman
...,...,...,...,...,...,...
627,Recording,NaN,Recording device,recorder/player,SFX,recorder/player
628,Field recording,NaN,N/A,NaN,AMB,unknown
629,Gramophone record,gramophone or vinyl record disc,c on turntable,Gramophone or vinyl record disc on a turntable,SFX,gramophone or vinyl record disc on a turntable
630,Compact disc,Compact Disc,c,digital audio Compact Disc,SFX,Compact Disc


In [37]:
df = df[['name','MajorityType','Object']]

In [38]:
df

,name,MajorityType,Object
0,Human sounds,SFX,human body
1,Human voice,SFX,Human
2,Speech,SFX,Human
3,"Male speech, man speaking",SFX,An adult male human
4,"Female speech, woman speaking",SFX,woman
...,...,...,...
627,Recording,SFX,recorder/player
628,Field recording,AMB,unknown
629,Gramophone record,SFX,gramophone or vinyl record disc on a turntable
630,Compact disc,SFX,Compact Disc


In [39]:
df.to_csv('data/augmented_labels.csv')

Some more manual curation

In [147]:
df = pd.read_csv('data/augmented_labels.csv', index_col=0)
onto = pd.read_json('data/ontology.json')[['name','description']]
df = pd.merge(df, onto, on='name', how='left')

In [148]:
# No object if ambience
df['Object'] = np.where(df['MajorityType']=='AMB', np.nan, df['Object'])

In [149]:
# Standardize "human" object
for row in df[df['MajorityType']=='SFX'].iterrows():
     index, series = row
     if series['Object'].lower() in ['human', 'individual', 'humans', 'person', 'an adult male human', 'human body', 'man', 'woman', 'face']:
         df.loc[df['name']==series['name'], 'Object'] = 'human'
     if series['Object'].lower() in ['cat', 'cats', 'a cat', 'cat or audience']:
         df.loc[df['name']==series['name'], 'Object'] = 'cat'
     if series['Object'].lower() in ['dog', 'dogs', 'a dog', 'miniature dog']:
         df.loc[df['name']==series['name'], 'Object'] = 'dog'
     if series['Object'].lower() in ['unknown', 'infinity symbol']:
         df.loc[df['name']==series['name'], 'Object'] = 'NaN'

In [186]:
# Speaking categories are of no interest for our purpose - we'll remove it
df['Exclude'] = 0 
for row in df.iterrows():
    index, series = row
    if 'speaking' in series['name'] or 'speech' in series['name'].lower() or 'human voice' in series['name'].lower() :
        df.loc[df['name']==series['name'], 'Exclude'] = 1

for row in df.iterrows():
    index, series = row
    if series['MajorityType'] == 'SFX':
        if series['Object'].lower() in ['guitar', 'ukulele', 'piano', 'organ', 'drum', 'maracas', 'keyboard'] or 'instrument' in series['Object'].lower() or 'drum' in series['Object'].lower() or 'piano' in series['Object'].lower() or 'organ' in series['Object'].lower():
            df.loc[df['name']==series['name'], 'MajorityType'] = 'MUS'
        if 'trumpet' in series['Object'].lower() or 'violin' in series['Object'].lower() or 'saxophone' in series['Object'].lower() or 'cello' in series['Object'].lower() or 'guitars' in series['Object'].lower():
            df.loc[df['name']==series['name'], 'MajorityType'] = 'MUS'
        if 'flute' in series['Object'].lower() or 'clarinet' in series['Object'].lower():
            df.loc[df['name']==series['name'], 'MajorityType'] = 'MUS'

    if 'music' in series['name'].lower() or 'music' in series['description'].lower() :
        df.loc[df['name']==series['name'], 'MajorityType'] = 'MUS'
    


In [187]:
## Manual curation
df.loc[df['name']=='Child speech, kid speaking', 'Object'] = 'Child'
df.loc[df['name']=='Fart', 'Object'] = 'human'

df.loc[df['name']=='Canidae, dogs, wolves', 'Object'] = 'Animal'
df.loc[df['name']=='Scissors', 'Object'] = 'Scissors'
df.loc[df['name']=='Typing', 'Object'] = 'Keyboard'
df.loc[df['name']=='Shuffling cards', 'Object'] = 'cards'
df.loc[df['name']=='Sound effect', 'Object'] = 'NaN'

df.loc[df['name']=='Gull, seagull', 'MajorityType'] = 'SFX'
df.loc[df['name']=='Gull, seagull', 'Object'] = 'bird'

df.loc[df['name']=='Owl', 'MajorityType'] = 'SFX'
df.loc[df['name']=='Owl', 'Object'] = 'bird'

df.loc[df['name']=='Insect', 'MajorityType'] = 'SFX'
df.loc[df['name']=='Insect', 'Object'] = 'insect'



df.loc[df['Object']=='computer or other machine', 'Object'] = 'Computer'


In [190]:
df['MajorityType'].value_counts()

MajorityType
SFX    401
MUS    179
AMB     52
Name: count, dtype: int64

In [189]:
df[df['MajorityType']=='SFX'].iloc[100:160]

,name,MajorityType,Object,description,Exclude
113,Bleat,SFX,sheep and goats,Communicative vocalization of sheep and goats.,0
114,Sheep,SFX,sheep,Sounds associated with the familiar quadrupeda...,0
115,Fowl,SFX,bird,Sounds associated with the bird species common...,0
116,"Chicken, rooster",SFX,Chicken,Sounds of the very common and widespread domes...,0
117,Cluck,SFX,chickens,"The soft, low-pitched, and repetitive sound ma...",0
118,"Crowing, cock-a-doodle-doo",SFX,rooster,The multi-syllable warning call made by a roos...,0
119,Turkey,SFX,bird,"Sounds associated with the large poultry bird,...",0
120,Gobble,SFX,turkey,The characteristic sound made by a turkey cock.,0
121,Duck,SFX,waterfowl species,Sounds associated with this collection of wate...,0
122,Quack,SFX,duck,The harsh communication sound made by a duck.,0


In [177]:
df['Object'].value_counts

<bound method IndexOpsMixin.value_counts of 0                                               human
1                                               human
2                                               human
3                                               human
4                                               human
                            ...                      
627                                   recorder/player
628                                               NaN
629    gramophone or vinyl record disc on a turntable
630                                      Compact Disc
631                 digital file using the MP3 format
Name: Object, Length: 632, dtype: object>